In [12]:
# 패키지 임포트
import requests
from bs4 import BeautifulSoup
import time
import urllib.request
from selenium.webdriver import Chrome
import re  # 정규표현식 패키지
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import datetime as dt
import pandas as pd

In [13]:
# 전체

In [22]:
delay = 3
path = "C:\dev\driver/chromedriver.exe"
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/69.0.3497.100 Safari/537.36'}
browser = Chrome(path)
# 딜레이를 주기 위함. delay = 3으로 설정,
browser.implicitly_wait(delay)

In [23]:
comment_data = pd.DataFrame({'youtube_id': [],
                             'comment': [],
                             'like_num': []})
browser = Chrome(path)  # chrome_options=options
#browser.implicitly_wait(delay)

In [24]:
# 검색어 입력 위해 youtube 접속
start_url = 'https://www.youtube.com'
browser.get(start_url)
# browser.maximize_window()

In [25]:
# 타다 금지법 메인 검색어 창에 입력
browser.find_elements_by_xpath(
    '//*[@id="search-form"]/div/div[1]')[0].click()  # 검색창영역클릭

browser.find_elements_by_xpath(
    '//*[@class="gstl_50 sbib_a"]/div[2]/input')[0].click()  # 검색창 영역 클립
browser.find_elements_by_xpath(
    '//*[@class="gstl_50 sbib_a"]/div[2]/input')[0].send_keys('타다 금지법')  # 타다 금지법 입력
browser.find_elements_by_xpath(
    '//*[@class="gstl_50 sbib_a"]/div[2]/input')[0].send_keys(Keys.RETURN)  # 엔터 검색 화면 전환

In [10]:
# 크롬 개발자 도구와 실제 파싱되어 나오는 html 코드가 다른 경우가 있다. get attribute를 우선 사용하는 것을 추천. 생성되지 않으면 일일이 코드를 뜯어봐서 찾아보기...

browser.find_elements_by_xpath(
    '//*[@id="container"]/ytd-toggle-button-renderer')[0].click()
browser.find_elements_by_xpath(
    '//*[@id="collapse-content"]/ytd-search-filter-group-renderer[1]/ytd-search-filter-renderer[3]/a')[0].click()

# browser.find_elements_by_xpath('//*[@id="collapse-content"]/ytd-search-filter-group-renderer[1]/ytd-search-filter-renderer[3]/a').send_keys(Keys.RETURN)

In [58]:
body = browser.find_element_by_tag_name('body')  # 스크롤하기 위해 소스 추출

num_of_pagedowns = 10
# 5번 밑으로 내리는 것
while num_of_pagedowns:
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(2)
    num_of_pagedowns -= 1

In [59]:
# 주소 파싱
html0 = browser.page_source
html = BeautifulSoup(html0, 'html.parser')

In [60]:
##### 여러가지 video_list1이 있지만 video_list1만 사용한다. 각 영상 주소 추출을 위한 것 #####
##### 나머지는 조회수, 추천수, 개시일등 확인 가능 #####

# <ytd-thumbnail class="style-scope ytd-video-renderer" use-hovered-property="">
# 검색에서 주소 가져오기

# 작동코드 건들지 말 것
video_list0 = html.find('div', {'id': 'contents'})
# video_list1 만 실제로 가져옴
video_list1 = html.find_all(
    'a', {'class': 'yt-simple-endpoint style-scope ytd-video-renderer'})
video_list1
#video_list1 = html.find_all('div', {'class': 'style-scope ytd-video-renderer'})
#video_list2 = video_list0.find_all('div', {'id': 'thumbnail'})
#video_list2 = video_list1.find_all('div', {'class': 'style-scope ytd-video-renderer'})
# video_list0[1].find('a')['href']
#title1 = video_list0[1].find('a',{'id':'video-title'}).text
# title1
# video_list1[0]['href']
# //*[@id="metadata-line"]/span[1]/text()
video_list2 = video_list0.find_all(
    'span', {'class': 'style-scope ytd-video-meta-block'})
video_list2[0]

##### 메타데이터를 사용하여 조회수 개시일 가져오기 text.split('/')[0] 사용하여 분리 #####
video_list3 = html.find_all(
    'div', {'class': 'text-wrapper style-scope ytd-video-renderer'})
video_list3[0].find('div', {'id': 'metadata-line'})

<div class="style-scope ytd-video-meta-block" id="metadata-line">
<span class="style-scope ytd-video-meta-block">조회수 2.1천회</span>
<span class="style-scope ytd-video-meta-block">6일 전</span>
<dom-repeat class="style-scope ytd-video-meta-block" strip-whitespace=""><template is="dom-repeat"></template></dom-repeat>
</div>

In [61]:
# 영상 주소 가져오기. 댓글, 좋아요 가져올 때 필요함
base_url = 'https://www.youtube.com'
test_url = []
for i in range(1, 30):
    url2 = base_url+video_list1[i]['href']
    test_url.append(url2)

In [62]:
# 영상 주소에서 첫 url을 시작 하도록
start_url = test_url[0]
browser.get(start_url)
body = browser.find_element_by_tag_name('body')

In [63]:
# 댓글 저장을 위해 빈  df 생성
comment_data = pd.DataFrame({'title': [],
                             'youtube_id': [],
                             'comment': [],
                             'like_num': []
                             })
browser = Chrome(path)  # chrome_options=options
browser.implicitly_wait(delay)

In [64]:
# 상위 2개의 영상의 세부 댓글만 가져오도록 설정 했음.

for i in range(1, 3):
    start_url = test_url[i]
    browser.get(start_url)
    body = browser.find_element_by_tag_name('body')

    time.sleep(2)  # 2초간 여유

    num_page_down = 1  # 페이지 다운도 적게
    while num_page_down:
        body.send_keys(Keys.PAGE_DOWN)
        time.sleep(1.5)
        num_page_down -= 1

        # 최신 댓글 순으로 가져오기 위한 설정
        # 댓글 정렬창 클릭
    browser.find_element_by_xpath(
        '//paper-button[@class="dropdown-trigger style-scope yt-dropdown-menu"]').click()
    # 최신 댓글 클릭
    browser.find_element_by_xpath(
        '//paper-listbox[@class="dropdown-content style-scope yt-dropdown-menu"]/a[1]').click()

    # 여러 댓글 가져오도록 스크롤
    num_page_down = 5  # 스크롤 2~3
    while num_page_down:
        body.send_keys(Keys.PAGE_DOWN)
        time.sleep(1.5)
        num_page_down -= 1

        # 파싱
    html_s0 = browser.page_source
    html_s = BeautifulSoup(html_s0, 'html.parser')

    # 제목 가져오기
    comment0 = html_s.find_all(
        'ytd-comment-renderer', {'class': 'style-scope ytd-comment-thread-renderer'})
    title0 = html_s.find_all(
        'ytd-video-primary-info-renderer')  # 제목 html 코드 가져오기
    title = title0[0].find('h1').text.split('/')[0]  # 제목 find하고 split

    # 영상별 댓글 가져오기
    for i in range(len(comment0)):

        # 댓글
        comment = comment0[i].find(
            'yt-formatted-string', {'id': 'content-text', 'class': 'style-scope ytd-comment-renderer'}).text

        # 좋아요 싫어요
        try:
            aa = comment0[i].find('span', {'id': 'vote-count-left'}).text

            like_num = "".join(re.findall('[0-9]', aa)) + "개"
        except:
            like_num = 0

        # 작성자 가져오기 정규 표현식으로 깔끔하게 가져오기.
        bb = comment0[i].find('a', {'id': 'author-text'}).find('span').text
        youtube_id = "".join(re.findall('[가-힣0-9a-zA-Z]', bb))

        # 한 영상에 대한 데이터 합치기
        insert_data = pd.DataFrame({'title': [title],
                                    'youtube_id': [youtube_id],
                                    'comment': [comment],
                                    'like_num': [like_num]
                                    })

        # 자료 붙여주기
        comment_data = comment_data.append(insert_data)

    # index 붙여주기
    comment_data.index = range(len(comment_data))

In [66]:
comment_data

,title,youtube_id,comment,like_num
0,'타다 금지법' 국회 본회의 통과,윤석,불법 콜택시 타다 사기꾼 이재웅 동등한 경쟁해야지 꽁짜로 쳐묵을려니까 문제지 ...,1개
1,'타다 금지법' 국회 본회의 통과,김영선,와이티엔은 타다편향 방송 안하고 공정하게 양측 울화통 터지지 않게 겸손한 방송 엄...,1개
2,'타다 금지법' 국회 본회의 통과,모모,난택시가 싢다 왜냐하면 좋은택시 기사도있었지만 난폭운전택시기사 끼어들기 u턴...,3개
3,'타다 금지법' 국회 본회의 통과,미스터손,"내 기억속의 택시는 기본요금 나왔다고 카드로 결제한다고 얼굴 찌푸렸던 택시, 물웅덩...",0개
4,'타다 금지법' 국회 본회의 통과,Kim노옹,택시 표심 겁나서 ㅎㅎㅎ,3개
5,'타다 금지법' 국회 본회의 통과,Frienze,중국애들 몰려드네,1개
6,'타다 금지법' 국회 본회의 통과,흠흠,저 이해 안 되는 데 요약 좀 해주실 분?,4개
7,'타다 금지법' 국회 본회의 통과,부엉이오형제,면허증따고 자격증따고 해라\n\n나? 면허없이 면허있는 파일럿보다 비행기 잘모는데 ...,5개
8,'타다 금지법' 국회 본회의 통과,부엉이오형제,국민의 다수가 원한다고 불법을 지양할순없는겁니다. 타다는 애당초 무허가콜택시사업이었...,7개
9,'타다 금지법' 국회 본회의 통과,Railroader,영 거꾸로 돌아가는듯한...,5개
